##  Let us begin

Can add descriptions later for users to load interactively. 


In [1]:
!nvidia-smi -L

GPU 0: NVIDIA RTX A6000 (UUID: GPU-69fba953-94a3-6b1e-c297-1c9b2d2713a9)


In [2]:
# import os
# import torch
# !pip install GPUtil

In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import json
import numpy as np
import argparse
import torch
import sys
import importlib as ipb
import pickle
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import GPUtil
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]
      
    def pickle(self, key_save):
        f = open(key_save, 'wb')
        pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)
        f.close()

    def unpickle(self, key_save):
        with open(key_save, 'rb') as f:
            return pickle.load(f)

    def add_result(self, run, result):
        assert len(result) == 2
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 0].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Test: {result[:, 1].max():.2f}')
            # Same as highest train, as we have no validation data
            print(f'  Final Train: {result[argmax, 0]:.2f}') 
            print(f'   Final Test: {result[argmax, 1]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)
            best_results = []
            for r in result:
                train = r[:, 0].max().item()
                test = r[:, 1].max().item()
                train2 = r[r[:, 0].argmax(), 0].item()
                test2 = r[r[:, 0].argmax(), 1].item()
                best_results.append((train, test, train2, test2))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Test: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')


def test(model, train_loader, test_loader):
    model.eval()
    loader = {0: train_loader, 1:test_loader}
    accuracies = []
    with torch.no_grad():
        for data_loader in loader.values():
            correct = 0
            total = 0
            for i, (images, labels) in enumerate(data_loader):
                if args.FC:
                    images = images.reshape(-1, input_size).to(device)
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            accuracies.append(correct/total)
    return accuracies # train_accu & test accu

### Training function (include SVI)

In [5]:
def train(model, images, labels):
    model.train()
    out = model(images)
    cost = nn.CrossEntropyLoss()
    loss = cost(out, labels)
    # loss = F.mse_loss(out.float(), F.one_hot(labels).float())/2
    loss.backward()
    return loss.item()


def train_SVI(model, images, labels):
    #### New lines for SVI ####
    # NOTE: lines below are necessary, as o/w model.layers_x grow in size as epoches increases
    model.layers_Xtilde = []
    model.layers_grad = []
    model.on_training = True
    #### End #####
    model.train()
    out = model(images)
    cost = nn.CrossEntropyLoss()
    loss = cost(out, labels)
    # loss = F.mse_loss(out.float(), F.one_hot(labels).float())/2
    loss.backward(retain_graph=True)
    #### New lines for SVI ####
    loss_tilde = 0
    for Xlplus1, Xlplus1grad in zip(reversed(model.layers_Xtilde), reversed(model.layers_grad)):
        Xlplus1grad = Xlplus1grad.grad.detach().to(device)
        loss_tilde += (Xlplus1*Xlplus1grad).sum()
    loss_tilde.backward()  # To get update direction by MVI for all layers at once
    model.on_training = False  # To avoid additional .retain_grad()
    #### End #####
    return loss.item()

### FCNet (include SVI)

In [6]:
class FCnet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(FCnet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(nn.Linear(hidden_channels, out_channels))
        self.dropout = dropout

    def forward(self, x):
        for i, conv in enumerate(self.convs):
            if i > 0:
                x = self.bns[i-1](x)
            x = conv(x)
            if i < len(self.convs)-1:
                x = F.relu(x)
            if i == len(self.convs[:-1])-1:
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x.softmax(dim=-1)


class FCnet_SVI(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(FCnet_SVI, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(nn.Linear(hidden_channels, out_channels))
        self.dropout = dropout
        #### New lines for SVI ####
        # If any layer true, then it needs to use SVI
        self.layers_with_SVI = [True for i in range(len(self.convs))]
        # TODO: later, can treat this as an input to be decided (as some layers need not SVI)
        # Append pre-activation \tilde{X}_{l+1}, ONLY at layers i where self.layers_with_SVI[i] == True
        self.layers_Xtilde = []
        # Append the grad of L w.r.t. X_{l+1}, ONLY at layers i where self.layers_with_SVI[i] == True
        self.layers_grad = []
        self.on_training = True # If False, then we do not use SVI.
        #### End #####

    def forward(self, x):
        for i, conv in enumerate(self.convs):
            if i > 0:
                x = self.bns[i-1](x)
            #### New lines for SVI ####
            # This is added b/c o/w the gradient backprop w.r.t. new loss also gets to earlier layers (undesirable)
            if self.layers_with_SVI[i] and self.on_training:
                x_tmp = x.detach().clone().to(device)
                self.layers_Xtilde.append(conv(x_tmp))
            #### End #####
            x = conv(x)
            if i < len(self.convs)-1:
                x = F.relu(x)
            if i == len(self.convs[:-1])-1:
                x = F.dropout(x, p=self.dropout, training=self.training)     
            #### New lines for SVI ####
            if self.layers_with_SVI[i] and self.on_training:
                x.retain_grad()  # To get the gradient with respect to output
                self.layers_grad.append(x)
            #### End #####
        return x.softmax(dim=-1)


### LeNet (include SVI)

In [7]:
class LeNet5(nn.Module):
    # From https://towardsdatascience.com/implementing-yann-lecuns-lenet-5-in-pytorch-5e05a0911320
    def __init__(self, in_channels, n_classes):
        super(LeNet5, self).__init__()

        # The convolutional filters are feature extractors
        self.convs = torch.nn.ModuleList()
        self.convs.append(nn.Conv2d(in_channels=in_channels,
                          out_channels=6, kernel_size=5, stride=1))
        self.convs.append(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1))
        self.convs.append(nn.Linear(in_features=400, out_features=120))
        self.convs.append(nn.Linear(in_features=120, out_features=84))
        self.convs.append(nn.Linear(in_features=84, out_features=n_classes))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm2d(6))
        self.bns.append(torch.nn.BatchNorm2d(16))

    def forward(self, x):
        for i, conv in enumerate(self.convs):
            if i == 2:
                x = torch.flatten(x, 1)
            x = conv(x)
            if i < 2: 
                x = self.bns[i](x)
            if i < len(self.convs) - 1:
                x = nn.ReLU()(x)
            if i < 2:
                # NOTE: This X_{l+1} is current input before Average Pooling, as o/w the \tilde{L} has dimension mismatch
                x = nn.AvgPool2d(2, stride=2)(x)
        return x


class LeNet5_SVI(nn.Module):
    def __init__(self, in_channels, n_classes):
        super(LeNet5_SVI, self).__init__()

        # The convolutional filters are feature extractors
        self.convs = torch.nn.ModuleList()
        self.convs.append(nn.Conv2d(in_channels=in_channels,
                          out_channels=6, kernel_size=5, stride=1))
        self.convs.append(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1))
        self.convs.append(nn.Linear(in_features=400, out_features=120))
        self.convs.append(nn.Linear(in_features=120, out_features=84))
        self.convs.append(nn.Linear(in_features=84, out_features=n_classes))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm2d(6))
        self.bns.append(torch.nn.BatchNorm2d(16))

        #### New lines for SVI ####
        # If any layer true, then it needs to use SVI
        self.layers_with_SVI = [True for i in range(len(self.convs))]
        # TODO: later, can treat this as an input to be decided (as some layers need not SVI)
        # Append pre-activation \tilde{X}_{l+1}, ONLY at layers i where self.layers_with_SVI[i] == True
        self.layers_Xtilde = []
        # Append the grad of L w.r.t. X_{l+1}, ONLY at layers i where self.layers_with_SVI[i] == True
        self.layers_grad = []
        self.on_training = True
        #### End #####

    def forward(self, x):
        for i, conv in enumerate(self.convs):
            if i == 2:
                x = torch.flatten(x, 1)
            #### New lines for SVI ####
            # This is added b/c o/w the gradient backprop w.r.t. new loss also gets to earlier layers (undesirable)
            if self.layers_with_SVI[i] and self.on_training:
                x_tmp = x.detach().clone().to(device)
                self.layers_Xtilde.append(conv(x_tmp))
            #### End #####
            x = conv(x)
            if i < 2: 
                x = self.bns[i](x)
            if i < len(self.convs) - 1:
                x = nn.ReLU()(x)
            #### New lines for SVI ####
            if self.layers_with_SVI[i] and self.on_training:
                x.retain_grad()  # To get the gradient with respect to output
                self.layers_grad.append(x)
            #### End #####
            if i < 2:
                # NOTE: This X_{l+1} is current input before Average Pooling, as o/w the \tilde{L} has dimension mismatch
                x = nn.AvgPool2d(2, stride=2)(x)
        return x

### Utility function, including data functions

In [8]:
def mem_report():
    if device.type == 'cuda':
        GPUs = GPUtil.getGPUs()
        for i, gpu in enumerate(GPUs):
            print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(
                i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    else:
        print("CPU RAM Free: "
              + humanize.naturalsize(psutil.virtual_memory().available))

In [9]:
import torchvision
import torchvision.transforms as transforms
def subset_data(data,frac=1):
    # Randomly subset a fraction of data from total data
    np.random.seed(1103)
    idx = np.random.choice(len(data),int(frac*len(data)),replace=False)
    return torch.utils.data.Subset(data,idx)
data_fixed = 'MNIST_batched' # 'MNIST_batched' or 'CIFAR10_batched'
if 'MNIST' in data_fixed:
    train_dataset0 = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)
    test_dataset0 = torchvision.datasets.MNIST(root = './data',
                                              train = False,
                                              transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                              download=True)
    num_classes = 10
    in_channels = 1
else:
    # NOTE, CIFAR10 has color channels, so input size = 3*32*32 with FC net
    train_dataset0 = torchvision.datasets.CIFAR10(root='/data', 
                                              train=True, 
                                              transform=transforms.ToTensor(),  
                                              download=True)
    test_dataset0 = torchvision.datasets.CIFAR10(root='/data', 
                                              train=False, 
                                              transform=transforms.ToTensor())
    num_classes = 10 
    in_channels = 3
input_size = torch.prod(torch.tensor(train_dataset0[0][0].shape)).item()
frac=0.1 if 'MNIST' in data_fixed else 0.2
frac = 1
train_dataset = subset_data(train_dataset0,frac=frac)
test_dataset = subset_data(test_dataset0,frac=frac)
print(len(train_dataset))
print(len(test_dataset))
# Data loader (i.e., split to batches) see below

60000
10000


Start testing

In [10]:
SVI_ls = [True, True, False]  # If False, use ordinary SGD or Adam
SVI_pause = [False, True, False]  # If True, only SVI to warm start
SVI_ls = [True]
SVI_pause = [True]
SVI_ls = [True]
SVI_pause = [False]
optim_ls = ['SGD', 'Adam']
lr, hidden_channels = 0.001, 512
FC_only = False  # Use all FC layers if True, o/w use LeNet
dataname = data_fixed
if FC_only == False:
    dataname = data_fixed + 'LeNet_'
num_runs = 3
batch_size = 128
num_epochs = 20
num_log_steps = len(train_dataset) // batch_size # Evaluate loss every X iteration/mini-batches
if __name__ == "__main__":
    Final_result = {}  # For quick check at the end of training
    for use_SVI, pause_SVI in zip(SVI_ls, SVI_pause):
        for optim_name in optim_ls:
            result_dict = {'SVI-SGD': [],
                           'SVI_warmstart-SGD': [],
                           'SGD': [],
                           'SVI-Adam': [],
                           'SVI_warmstart-Adam': [],
                           'Adam': []}
            parser = argparse.ArgumentParser(
                description=dataname)
            parser.add_argument('--log_steps', type=int, default=num_log_steps)
            parser.add_argument('--num_layers', type=int, default=4)
            parser.add_argument('--dropout', type=float, default=0.25)
            parser.add_argument('--lr', type=float, default=lr)
            parser.add_argument('--momentum', type=float, default=0.95)
            parser.add_argument(
                '--epochs', type=int, default=num_epochs)  # Change to 100
            parser.add_argument(
                '--batch_size', type=int, default=batch_size)
            parser.add_argument('--runs', type=int,
                                default=num_runs)
            parser.add_argument('--SVI', type=bool, default=use_SVI)
            parser.add_argument('--FC', type=bool, default=FC_only)
            parser.add_argument(
                '--optimizer', type=str, default=optim_name)
            args = parser.parse_args(args=[])
            args.hidden_channels = hidden_channels
            print(args)
            # Get data loader from dataset
            train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                       batch_size=batch_size,
                                                       shuffle=True)
            test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                      batch_size=batch_size,
                                                      shuffle=False)
            logger = Logger(args.runs, args)
            results_over_runs = {}
            for run in range(args.runs):
                accu_at_run = []
                # Initiatiate new model per run
                torch.manual_seed(1103 + run)
                args.SVI = use_SVI
                if args.SVI:
                    if args.FC:
                        model = FCnet_SVI(input_size, args.hidden_channels,
                                          num_classes, args.num_layers, args.dropout).to(device)
                    else:
                        model = LeNet5_SVI(in_channels, num_classes).to(device)
                else:
                    if args.FC:
                        model = FCnet(input_size, args.hidden_channels,
                                      num_classes, args.num_layers, args.dropout).to(device)
                    else:
                        model = LeNet5(in_channels, num_classes).to(device)
                if args.optimizer == 'SGD':
                    optimizer = torch.optim.SGD(
                        model.parameters(), lr=args.lr, momentum=args.momentum, nesterov=True)
                else:
                    optimizer = torch.optim.Adam(
                        model.parameters(), lr=args.lr)
                for epoch in range(1, 1 + args.epochs):
                    iter_per_epoch = int(len(train_dataset) / batch_size) + 1
                    frac_stop = 0.1
                    pause_threshold = int(
                        frac_stop * iter_per_epoch * args.epochs)
                    if device.type == 'cuda':
                        # Useful to avoid GPU allocation excess
                        torch.cuda.empty_cache()
                    print(f"LR is {optimizer.param_groups[0]['lr']}")
                    for i, (images, labels) in enumerate(train_loader):
                        current_itr = ((epoch - 1) * iter_per_epoch) + i + 1
                        if pause_SVI and current_itr > pause_threshold:
                            args.SVI = False
                            if current_itr <= pause_threshold + 1:
                                # Reinitialize optimizer to avoid gradient issue in Adam
                                sdict = model.state_dict()
                                print(
                                    '############ Pause SVI from now on ############')
                                if args.FC:
                                    model = FCnet(input_size, args.hidden_channels,
                                                  num_classes, args.num_layers, args.dropout)
                                else:
                                    model = LeNet5(in_channels, num_classes)
                                model.load_state_dict(sdict)
                                model = model.to(device)
                                optimizer = torch.optim.Adam(
                                    model.parameters(), lr=args.lr)
                        optimizer.zero_grad()
                        if args.FC:
                            images = images.reshape(-1, input_size).to(device)
                        images = images.to(device)
                        labels = labels.to(device)
                        loss = train_SVI(model, images, labels) if args.SVI else train(model, images, labels)
                        optimizer.step()
                        # Test at each iteration
                        if (i+1) % args.log_steps == 0:
                            print('Testing')
                            if args.SVI:
                                print(f'SVI-{args.optimizer} training at batch {i}, epoch {epoch}')
                            else:
                                print(f'{args.optimizer} training at batch {i}, epoch {epoch}')
                            # Do so because training data has too many images, and evaluation thus takes too long.
                            train_loader_sub = torch.utils.data.DataLoader(dataset=subset_data(train_dataset0,frac=0.1),
                                                       batch_size=batch_size,
                                                       shuffle=True)
                            result = test(model, train_loader_sub, test_loader)
                            logger.add_result(run, result)
                            train_acc, test_acc = result
                            accu_at_run += [[train_acc, test_acc]]
                            print(f'Run: {run + 1:02d}, '
                                  f'Epoch: {epoch:02d}, '
                                  f'Loss: {loss:.4f}, '
                                  f'Train: {100 * train_acc:.2f}%,'
                                  f'Test: {100 * test_acc:.2f}%')
                # Save model after training over all epochs
                results_over_runs[f'lr={args.lr}@Run{run+1}'] = accu_at_run
                logger.print_statistics(run)
                # Save results
                if use_SVI:
                    SVI_prefix = 'SVI_warmstart-' if pause_SVI else 'SVI-'
                else:
                    SVI_prefix = ''
                key = f'{SVI_prefix}{optim_name}'
                key_save = f'{SVI_prefix}{optim_name}-{args.num_layers}layers-{args.hidden_channels}nodes-{args.lr}LR'
                key_save = dataname + key_save
                # Save it to file, but need not now because only one run.
                logger.pickle(key_save)
                result_dict[key].append(results_over_runs)
                # Final train and test accuracy
                Final_result[key] = accu_at_run[-1]
                with open(f"{key_save}_loss_together.json", "w") as outfile:
                    json.dump(result_dict, outfile)
            logger.print_statistics()

Namespace(log_steps=468, num_layers=4, dropout=0.25, lr=0.001, momentum=0.95, epochs=20, batch_size=128, runs=3, SVI=True, FC=False, optimizer='SGD', hidden_channels=512)
LR is 0.001
Testing
SVI-SGD training at batch 467, epoch 1
Run: 01, Epoch: 01, Loss: 0.1810, Train: 95.10%,Test: 95.27%
LR is 0.001
Testing
SVI-SGD training at batch 467, epoch 2
Run: 01, Epoch: 02, Loss: 0.1699, Train: 97.25%,Test: 97.33%
LR is 0.001
Testing
SVI-SGD training at batch 467, epoch 3
Run: 01, Epoch: 03, Loss: 0.0495, Train: 97.35%,Test: 97.74%
LR is 0.001
Testing
SVI-SGD training at batch 467, epoch 4
Run: 01, Epoch: 04, Loss: 0.1093, Train: 97.85%,Test: 98.09%
LR is 0.001
Testing
SVI-SGD training at batch 467, epoch 5
Run: 01, Epoch: 05, Loss: 0.0689, Train: 98.17%,Test: 98.38%
LR is 0.001
Testing
SVI-SGD training at batch 467, epoch 6
Run: 01, Epoch: 06, Loss: 0.0323, Train: 98.28%,Test: 98.50%
LR is 0.001
Testing
SVI-SGD training at batch 467, epoch 7
Run: 01, Epoch: 07, Loss: 0.0705, Train: 98.50%,Te